In [1]:
# https://joint-research-centre.ec.europa.eu/pvgis-online-tool/getting-started-pvgis/api-non-interactive-service_en

import os, csv, json, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import clear_output

from pathlib import Path
pd.options.display.float_format = '{:.2f}'.format

### Parameters

In [2]:
startyear     = 2020
endyear       = 2020
peakpower     = 1
loss          = 14
pvcalculation = 1 # "0" outputs only solar radiation calculations, "1" outputs the estimation of hourly PV production as well
optimalangles = 1 #  Value of 1 for "yes". All other values (or no value) mean "no". 
localtime     = 1 # Output the time in the local time zone (not daylight saving time), instead of UTC. Value of 1 for "yes". All other values (or no value) mean "no". Daylight saving time is NOT taken into account.

# excel_filename = 'SLE_results1.13.xlsx'

excel_filename = "TMP_test_Congo.xlsx"

#### it seems local time does not work

### File names

In [3]:
home        = Path(os.getcwd())
xls_file    = home.joinpath(excel_filename)
csv_outfile = home.joinpath(f'{xls_file.stem}_PVGIS_year{startyear}to{endyear}.csv')


### pandas

In [14]:
df_xls = pd.read_excel(xls_file)

columns = ['idSet','latitude','longitude']
df = df_xls[columns].copy()

# df = df_xls[columns].head(2).copy() # test only

print(df.head(10))

   idSet  latitude  longitude
0      1      0.80      19.20
1      2     -2.20      17.06
2      3     -5.45      17.50
3      4     -1.80      19.92
4      5      1.89      21.05
5      6      2.42      26.83
6      7      0.35      28.93
7      8     -4.86      26.89
8      9     -5.71      23.32
9     10    -10.23      26.09


### create urls

In [15]:
def get_url(lat, lon):    
    
    url_base = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?"
    
    pvgis_params = dict(
        peakpower=peakpower,
        loss=loss,
        localtime=localtime,
        startyear=startyear,
        endyear=endyear,
        pvcalculation=pvcalculation, # "0" outputs only solar radiation calculations, "1" outputs the estimation of hourly PV production as well
        optimalangles=optimalangles,
        lat=lat,
        lon=lon,
        outputformat = 'json',
    )   

    return url_base + "&".join([f'{key}={value}' for key, value in pvgis_params.items()])


In [16]:
def parse_json(id, data, csv_file):    
    
    # ____________ parse data to df
    df_input        = pd.json_normalize(data.get('inputs'))
    df_output       = pd.json_normalize(data.get('outputs').get('hourly'))        
    latitude        = df_input['location.latitude'].values
    
    # ____________ get optimal angles
    slope_optimized   = df_input['mounting_system.fixed.slope.value'].values
    azimuth_optimized = df_input['mounting_system.fixed.azimuth.value'].values    
    azimuth_corrected = 0 if float(latitude) < 0 else 180

    # ____________ convert the 'Date' column to datetime format: round minutes from timestamp (do not trunc the timestamp - it would cause a shift in some readings)
    df_power               = df_output[['time', 'P']].copy() 
    df_power['time']       = pd.to_datetime(df_power['time'], format='%Y%m%d:%H%M').round('H').dt.strftime('%Y-%m-%d:%H')   
    
    # ____________ set datetime index
    df_power = df_power.set_index(df_power['time'])
    
    # ____________ transpose df
    df_tranpose = df_power[['P']].transpose()   
    
    # ____________ add columns
    df_tranpose.insert(0, 'azimuth_corrected', azimuth_corrected)
    df_tranpose.insert(0, 'azimuth', azimuth_optimized)
    df_tranpose.insert(0, 'slope', slope_optimized)
    df_tranpose.insert(0, 'id', id)
    
    return df_tranpose   
    

### query PVGIS

In [18]:
i = 0
dfs = dict()

for idx, row in df.iterrows(): 
    
    id = row['idSet'].astype(int)
    lat = row['latitude'].astype(str)
    lon = row['longitude'].astype(str)
    url = get_url (lat,lon)
    
    csv_file = home.joinpath('csv', f"SLE_{id}.csv")
    
    print (url)

    response = requests.get(url)
    row_json = json.loads(response.text)    

    try:            
        dfs[id] = parse_json(id, row_json, csv_file) #  parse data from pvgis
    except:
        dfs[id] = None
        print (f'error parse json id: {id}, lat:{lat}, lon:{lon}')

    # _____________ save partial results
    if i > 100:  
        print ('_____________________________')
        df_merged = pd.concat(dfs.values(), ignore_index=True)
        df_merged.to_csv(csv_outfile, index = False)
        i = 0
        clear_output(wait=False)

    i +=1
            

# _____________ save final results
df_merged = pd.concat(dfs.values(), ignore_index=True)
df_merged.to_csv(csv_outfile, index = False)


https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&localtime=1&startyear=2020&endyear=2020&pvcalculation=1&optimalangles=1&lat=0.8&lon=19.198&outputformat=json
https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&localtime=1&startyear=2020&endyear=2020&pvcalculation=1&optimalangles=1&lat=-2.201&lon=17.059&outputformat=json
https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&localtime=1&startyear=2020&endyear=2020&pvcalculation=1&optimalangles=1&lat=-5.449&lon=17.5&outputformat=json
https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&localtime=1&startyear=2020&endyear=2020&pvcalculation=1&optimalangles=1&lat=-1.801&lon=19.921&outputformat=json
https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&localtime=1&startyear=2020&endyear=2020&pvcalculation=1&optimalangles=1&lat=1.893&lon=21.045&outputformat=json
https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&localtime=1&startyear=2020&endyear=202